In [1]:
SBOX = [0, 6, 14, 1, 15, 4, 7, 13, 9, 8, 12, 5, 2, 10, 3, 11]

valid_pairs = set()
for a in range(16):
    for b in range(16):
        dx = a ^ b
        dy = SBOX[a] ^ SBOX[b]
        valid_pairs.add((dx, dy))

# make a binary matrix
import numpy as np
M = np.zeros((16,16), dtype=int)
for (dx,dy) in valid_pairs:
    M[dx,dy] = 1
print(M)


[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 1 0 1 1 1 1 0 0 0 1]
 [0 1 0 0 0 1 0 1 1 1 0 0 0 1 1 0]
 [0 1 1 1 1 0 0 0 1 1 0 0 1 0 0 0]
 [0 0 1 0 0 0 0 0 0 1 0 1 1 0 1 1]
 [0 0 0 1 1 0 1 1 0 1 1 0 0 0 0 0]
 [0 1 0 1 0 1 0 1 0 0 1 1 0 0 1 1]
 [0 1 1 0 0 0 1 1 0 0 1 1 0 1 1 0]
 [0 0 1 0 1 0 1 0 0 1 0 0 0 1 1 0]
 [0 0 0 0 0 1 1 0 1 0 0 1 1 1 1 1]
 [0 0 0 1 0 1 0 1 0 1 0 0 1 0 0 1]
 [0 0 0 0 1 1 1 1 1 0 1 0 0 1 0 1]
 [0 0 1 0 0 0 1 0 1 0 1 1 1 1 0 0]
 [0 1 1 0 1 1 0 1 0 0 1 0 0 1 0 0]
 [0 1 0 1 0 1 0 0 0 0 0 1 1 1 1 0]
 [0 0 0 1 1 1 0 0 1 0 1 1 0 0 0 1]]


In [3]:
# ============================================================
# Toy Saturnin S-box Convex Hull → Reduced Inequalities → MILP
# ============================================================

# --- Step 0. Imports ---
from sage.all import *
from itertools import product

# ============================================================
# 1️⃣ Define your S-box
# ============================================================
SBOX = [0, 6, 14, 1, 15, 4, 7, 13, 9, 8, 12, 5, 2, 10, 3, 11]
BITS = 4

# ============================================================
# 2️⃣ Generate all valid (Δx, Δy) points
# ============================================================
valid_pts = []
for a in range(16):
    for b in range(16):
        dx = a ^ b
        dy = SBOX[a] ^ SBOX[b]
        # Convert to binary vector [x3,x2,x1,x0,y3,y2,y1,y0]
        xbits = [(dx >> i) & 1 for i in reversed(range(BITS))]
        ybits = [(dy >> i) & 1 for i in reversed(range(BITS))]
        valid_pts.append(xbits + ybits)

print(f"[+] {len(valid_pts)} valid (Δx,Δy) points out of 256 total")

# ============================================================
# 3️⃣ Build convex hull polyhedron and extract inequalities
# ============================================================
P = Polyhedron(vertices=valid_pts)
ineqs = P.Hrepresentation()

print(f"[+] Convex hull has {len(ineqs)} inequalities")

# Convert inequalities into integer (coeffs, rhs) form
H = []
for ineq in ineqs:
    # ineq is of form a0*x0 + a1*x1 + ... + c >= 0
    coeffs = [QQ(a) for a in ineq.A()]  # coefficients
    rhs = -QQ(ineq.b())                 # move to <= form (A*x <= b)
    # Normalize to integers
    denom = lcm([c.denominator() for c in coeffs] + [rhs.denominator()])
    coeffs_int = [int(c * denom) for c in coeffs]
    rhs_int = int(rhs * denom)
    H.append((coeffs_int, rhs_int))

print(f"[+] Converted to {len(H)} integer inequalities.")

# ============================================================
# 4️⃣ Prepare impossible patterns X
# ============================================================
valid_set = {tuple(p) for p in valid_pts}
all_patterns = list(product([0,1], repeat=8))
X = [p for p in all_patterns if p not in valid_set]

print(f"[+] {len(X)} impossible patterns to eliminate.")

# ============================================================
# 5️⃣ Greedy selection of top-n inequalities
# ============================================================
def killed_by(ineq, patterns):
    coeffs, rhs = ineq
    killed = []
    for p in patterns:
        val = sum(c * bit for c, bit in zip(coeffs, p))
        if val > rhs:
            killed.append(p)
    return killed

def select_inequalities(H, X, n):
    H_remaining = H[:]
    X_remaining = set(X)
    chosen = []

    for _ in range(n):
        if not X_remaining:
            break
        best = None
        best_killed = set()
        for ineq in H_remaining:
            killed = set(killed_by(ineq, X_remaining))
            if len(killed) > len(best_killed):
                best_killed = killed
                best = ineq
        if best is None:
            break
        chosen.append(best)
        X_remaining -= best_killed
        H_remaining.remove(best)
        print(f"[i] Picked ineq {_+1}, killed {len(best_killed)} patterns, remaining {len(X_remaining)}")

    return chosen

# Pick top-n inequalities (you can tune n)
N_SELECT = 15
H_selected = select_inequalities(H, X, N_SELECT)
print(f"[+] Selected {len(H_selected)} inequalities")

# ============================================================
# 6️⃣ Export selected inequalities to Python format
# ============================================================
with open("sbox_hull_ineqs.py", "w") as f:
    f.write("# Auto-generated from Sage convex hull\n")
    f.write("INEQUALITIES = [\n")
    for coeffs, rhs in H_selected:
        f.write(f"    ({coeffs}, {rhs}),\n")
    f.write("]\n")
print("[+] Saved reduced inequalities to sbox_hull_ineqs.py")


[+] 256 valid (Δx,Δy) points out of 256 total
[+] Convex hull has 222 inequalities
[+] Converted to 222 integer inequalities.
[+] 150 impossible patterns to eliminate.
[i] Picked ineq 1, killed 149 patterns, remaining 1
[i] Picked ineq 2, killed 1 patterns, remaining 0
[+] Selected 2 inequalities
[+] Saved reduced inequalities to sbox_hull_ineqs.py
